### Установка и импорт необходимых зависимостей

In [ ]:
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install matplotlib>=3.3.2

BRANCH = 'r1.21.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

In [ ]:
import torch
from omegaconf import OmegaConf
import torchaudio

from nemo.collections.asr.models import EncDecCTCModel

from nemo.collections.asr.parts.preprocessing.features import FilterbankFeaturesTA as NeMoFilterbankFeaturesTA
from nemo.collections.asr.modules.audio_preprocessing import AudioToMelSpectrogramPreprocessor as NeMoAudioToMelSpectrogramPreprocessor

### Загрузка конфига модели, ее весов и аудио для примера

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# Loading weights, config and example wav for CTC-model
!wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/ctc_model_weights.ckpt
!wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/ctc_model_config.yaml

### Добавление модулей для извлечения фичей, подробнее о которых можно узнать по следующей ссылке https://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html

In [ ]:
from typing import Optional, Tuple, Union
import random

class FilterbankFeaturesTA(NeMoFilterbankFeaturesTA):
    def __init__(self, mel_scale: str = 'htk', wkwargs=None, **kwargs):
        if 'window_size' in kwargs:
            del kwargs['window_size']
        if 'window_stride' in kwargs:
            del kwargs['window_stride']

        super().__init__(**kwargs)

        self._mel_spec_extractor: torchaudio.transforms.MelSpectrogram = torchaudio.transforms.MelSpectrogram(
            sample_rate=self._sample_rate,
            win_length=self.win_length,
            hop_length=self.hop_length,
            n_mels=kwargs['nfilt'],
            window_fn=self.torch_windows[kwargs['window']],
            mel_scale=mel_scale,
            norm=kwargs['mel_norm'],
            n_fft=kwargs['n_fft'],
            f_max=kwargs.get('highfreq', None),
            f_min=kwargs.get('lowfreq', 0),
            wkwargs=wkwargs,
        )

class AudioToMelSpectrogramPreprocessor(NeMoAudioToMelSpectrogramPreprocessor):
    def __init__(self, mel_scale: str = 'htk', **kwargs):
        super().__init__(**kwargs)
        kwargs['nfilt'] = kwargs['features']
        del kwargs['features']
        self.featurizer = FilterbankFeaturesTA(  # Deprecated arguments; kept for config compatibility
            mel_scale=mel_scale,
            **kwargs,
        )

### Пример использования ммодели CTC для транскрибации

In [ ]:
model = EncDecCTCModel.from_config_file('./ctc_model_config.yaml') # загрузка конфига модели
ckpt = torch.load('./ctc_model_weights.ckpt', map_location='cpu') # загрузка весов модели
model.load_state_dict(ckpt, strict=False)
model.eval() # перевод модели в стадию инференса
model = model.to('cuda') # перевод модели на GPU

### Подготовка данных:
### 1. Импорт необходимых библиотек
### 2. Разархивирование zip-файла со всеми аудио
### 3. Создание пустых массивов для загрузки в них данных

In [ ]:
import torchaudio
import numpy as np
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence
import zipfile

In [ ]:
zip_ref = zipfile.ZipFile("audios1.zip", 'r') # извлечение файла с записями диалогов
zip_ref.extractall("audios1")
zip_ref.close()

In [ ]:
audios = []
sentences = []
used = []

## Сопоставление каждому из аудио его транскрибацию для создания датасета

In [ ]:
for file in os.listdir('audios1/audios1'):

  audio = AudioSegment.from_mp3('audios1/audios1/' + file)

  chunks = split_on_silence (
    audio,
    min_silence_len = 1000,
    silence_thresh = -50
  )

  for i, ch in enumerate(chunks):
    if i != 0:
      path = 'audios1/forusage' + str(i) + '.wav'
      ch.export(path, format='wav')
      audios.append(torchaudio.load(path)[0])
      sentences.append(model.transcribe([path]))
    for path in os.listdir('audios1/forusage'):
      os.remove('audios1/forusage' + path)

  used.append(file)

In [ ]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

d = {'train':Dataset.from_dict({'audio':audios,'sentence':sentences}),
     }

data = DatasetDict(d) # создание нужного класса DatasetDict для дальнейшей выгрузки датасета на HuggingFace

In [ ]:
data.push_to_hub(repo_id='for_whisper_ft', private=True) # добавление датасета на HuggingFace

### Проверка возможности загрузить данные обратно в код

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

dataset = load_dataset("vladimir7542/for_whisper_ft1")

In [ ]:
def prepare_dataset(batch): # функция подготовки данных для дальнешего обучения модели транскрибации

    batch["input_features"] = feature_extractor(batch["audio"], sampling_rate=16000).input_features[0]

    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=2)